In [1]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt 
import xarray as xr
import glob
import numpy as np
import os
%load_ext tensorboard

In [4]:
from sclouds.helpers import path_

SyntaxError: invalid syntax (<ipython-input-4-6a7edb8d4c76>, line 1)

In [3]:
# Hyperparameters useful stuff - 
num_epochs = 4
batch_size = 32
# learning_rate = 0.001

read_path  = '/home/hanna/lagrings/ERA5_monthly/'
write_path = '/home/hanna/lagrings/results/convlstm/'

In [33]:
def dataset_to_numpy(pixel, bias = True):
    """ Dataset. """ 
    n     = len(pixel.time.values)
    n_lat = len(pixel.latitude.values)
    n_lon = len(pixel.longitude.values)
    
    if bias:
        num_vars = 4+1
    else:
        num_vars = 4 
    
    X = np.zeros((n, n_lat, n_lon, num_vars))
    
    q   = pixel.q.values
    t2m = pixel.t2m.values
    r   = pixel.r.values
    sp  = pixel.sp.values
    tcc = pixel.tcc.values

    X[:, :, :, 0] = q
    X[:, :, :, 1] = t2m
    X[:, :, :, 2] = r
    X[:, :, :, 3] = sp
    
    if bias: # LSTM network should not input a bias
        X[:, :, :, 4] = 1
    
    return X, tcc[:, :, :, np.newaxis]

In [34]:
def load_data(year = None, month = None):
    """
    Currently limited to one year or one month, or the entire period.
    """
    if not year:
        year = '*'
    
    if not month:
        month = '*'
    else:
        month = "%2.2d" %month
    
    #if not variable:
    #    variable = '*'
    
    search_str = '{}_{}_*.nc'.format(year, month)
    
    write_path = '/uio/lagringshotell/geofag/students/metos/hannasv/ERA5_monthly/'
    read_path = '/uio/lagringshotell/geofag/students/metos/hannasv/dataERA5_grib/'

    base = '/home/hanna/lagrings'
    read_folder = 'ERA5_monthly'
    
    folder = os.path.join(base, read_folder)
    files = glob.glob(os.path.join(folder, search_str))
    data = [xr.open_dataset(fil) for fil in files]
    all_data = xr.merge(data, 'equals')
    X, y = dataset_to_numpy(all_data, bias = False)
    return X, y

In [35]:
X, y = load_data(year = 2005, month = 5, variable = None)

<xarray.Dataset>
Dimensions:        (latitude: 81, longitude: 161, time: 744)
Coordinates:
  * latitude       (latitude) float64 30.0 30.25 30.5 30.75 ... 49.5 49.75 50.0
  * time           (time) datetime64[ns] 2005-05-01 ... 2005-05-31T23:00:00
    number         int64 0
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 1000
  * longitude      (longitude) float64 -15.0 -14.75 -14.5 ... 24.5 24.75 25.0
    valid_time     (time) datetime64[ns] 2005-05-01 ... 2005-05-31T23:00:00
    surface        int64 0
Data variables:
    r              (time, latitude, longitude) float32 ...
    t2m            (time, latitude, longitude) float32 ...
    q              (time, latitude, longitude) float32 ...
    sp             (time, latitude, longitude) float32 ...
    tcc            (time, latitude, longitude) float64 0.0 0.0 0.0 ... 1.0 1.0
    nr_nans        (time, latitude, longitude) float64 0.0 0.0 0.0 ... 0.0 0.0
(744, 81, 161)


In [74]:
np.isnan(X).sum()

0

In [37]:
def train_test_split(data, test_split = 0.2):
    assert test_split < 1, 'test split is given as a decimal number, choose a number between 0, 1'
    indx = int(len(data)*(1-test_split))
    # returns train test
    return data[:indx], data[indx:]

In [38]:
def batch_normalize(data, seq_len = 4): 
    samples, metvars, lats, lons = data.shape
    #.mean(axis = 0).mean(axis=1).mean(axis=1)
    
    normalized = np.zeros((samples, metvars, lats, lons))
    means   = np.zeros(metvars)
    storage = np.zeros(metvars)
    
    for i in range(metvars):
        raveled = data[:, i, :, :].reshape(-1)
        m = raveled.mean()
        s = raveled.std()
        normalized[:, i, :, :] =  (data[:, i, :, :] - m)/s
        
    samples, metvars, lats, lons = normalized.shape
    assert seq_len % 4 == 0
    
    new_samples = int(samples/seq_len)
    normalized  = normalized.reshape( (new_samples, seq_len, metvars, lats, lons ) )
    
    return normalized, means, storage

In [39]:
a, d, g = batch_normalize(X)

In [40]:
a.shape

(186, 4, 81, 161, 4)

In [41]:
from tensorflow.keras.layers import ConvLSTM2D
#from keras.layers.normalization import BatchNormalization

In [42]:
x, means_x, storage_x = batch_normalize(X)

In [43]:
y, means_y, storage_y = batch_normalize(y)

In [76]:
np.isnan(y).sum()

9702504

In [77]:
186*4*81*161

9702504

## Normalize

In [18]:
#x = x.reshape(-1, 2184, 4, 35, 60)
#y = y.reshape(-1, 2184, 1, 35, 60)

In [64]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import losses
from tensorflow.keras import optimizers

#from keras.layers import keras.layers.

seq =  Sequential()

# Begin with 2D convolutional LSTM layer
seq.add(ConvLSTM2D(filters = 256, kernel_size = (3, 3),
                     input_shape = (4, 81, 161, 4),
                     padding = 'same', return_sequences=True, data_format='channels_last')) 

seq.add(keras.layers.ConvLSTM2D(filters = 128, kernel_size = (3, 3),
                                #input_shape = (256, 35, 60, 1),
                                padding='same', return_sequences=True,  data_format='channels_last')) 

seq.add(keras.layers.ConvLSTM2D(filters=64, kernel_size=(3, 3),
                                  #input_shape=(None, 35, 60, 1),
                                  padding='same', return_sequences=True,  data_format='channels_last')) 

# Add 3x hidden 2D convolutions LSTM layers    
# Begin with 2D convolutional LSTM layer
seq.add(keras.layers.ConvLSTM2D(filters=1, kernel_size=(1, 1), # (1, 1)
                                #input_shape=(None, 40, 40, 1),
                                padding='same', return_sequences=True,  data_format='channels_last')) 


In [72]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
seq.compile(loss='mean_squared_error', optimizer=sgd)


In [73]:
history = seq.fit(x=x, y=y, batch_size=1, epochs=1, verbose=1, callbacks=None, 
                  validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, 
                  sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, 
                  validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)


Train on 186 samples
 27/186 [===>..........................] - ETA: 49:56 - loss: nan

KeyboardInterrupt: 

In [48]:
def plot(history):
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    return

In [22]:
#from keras.utils import plot_model
#plot_model(model, to_file=save_dir+'model.png')

In [ ]:
# Prepare model for training
model1.compile(
      loss = "mse",
      metrics = ["mae"],
      optimizer = "adam"
)
import datetime
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


history = model1.fit(x,
                    y,
                    batch_size = 32,
                    epochs = 4,
                    validation_split = 0.2,
                    callbacks=[tensorboard_callback]
                    )

plot(history)

Train on 436 samples, validate on 110 samples
Epoch 1/4


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
history

In [2]:
from tensorkeras.models import Sequential
from keras.layers import ConvLSTM2D
#from keras.layers.normalization import BatchNormalization

#from keras.layers import keras.layers.

def build_model(filters = [256, 128, 64], kernal = [5, 5, 3], input_shape = (4, 4, 35, 60)):
    model =  Sequential()

    # Begin with 2D convolutional LSTM layer
    for i, filter_ in enumerate( filters ):
        k_s = kernal[i]
        
        if i == 0:
            model.add(ConvLSTM2D(filters = filter_, kernel_size = (k_s, k_s),
                                 input_shape = input_shape,
                                 padding = 'same', return_sequences=True, data_format='channels_first')) 
        else:
            model.add(ConvLSTM2D(filters = filter_, kernel_size =  (k_s, k_s),
                                              #input_shape = (256, 35, 60, 1),
                                              padding='same', return_sequences=True,  data_format='channels_first')) 

    # Add 3x hidden 2D convolutions LSTM layers    
    # Begin with 2D convolutional LSTM layer
    # Blir dette rett da..?
    model.add(keras.layers.ConvLSTM2D(filters=1, kernel_size=(1, 1),
                                      #input_shape=(None, 40, 40, 1),
                                      padding='same', return_sequences=True,  data_format='channels_first')) 
    return model

ModuleNotFoundError: No module named 'keras'

In [1]:


model =  Sequential()

# Begin with 2D convolutional LSTM layer
model.add(ConvLSTM2D(filters = 256, kernel_size = (5, 5),
                     input_shape = (4, 4, 35, 60),
                     padding = 'same', return_sequences=True, data_format='channels_first')) 

model.add(keras.layers.ConvLSTM2D(filters = 128, kernel_size = (5, 5),
                                  #input_shape = (256, 35, 60, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

model.add(keras.layers.ConvLSTM2D(filters=64, kernel_size=(3, 3),
                                  #input_shape=(None, 35, 60, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

# Add 3x hidden 2D convolutions LSTM layers    
# Begin with 2D convolutional LSTM layer
model.add(keras.layers.ConvLSTM2D(filters=1, kernel_size=(3, 3),
                                  #input_shape=(None, 40, 40, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

# Prepare model for training
model.compile(
      loss = "mse",
      metrics = ["mae"],
      optimizer = "adam"
)

history2 = model.fit(x,
                    y,
                    batch_size = 32,
                    epochs = 40,
                    validation_split = 0.2, 
                    callbacks=[tensorboard_callback]
                    )



ModuleNotFoundError: No module named 'keras'

In [ ]:
history2

In [ ]:
plot(history2)

In [ ]:
from keras.models import model_from_json


def save_json(model, m = "model.json"):    
    """ Storing a config / architecture of the model."""
    # serialize model to JSON
    model_json = model.to_json()
    with open(m, "w") as json_file:
        json_file.write(model_json)
    return

def load_model(m = 'model.json'):
    """ Loading a stored config/ architecture  to a model. """
    # load json and create model
    json_file = open(m, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    return loaded_model

In [ ]:
model_storage = '/home/hanna/MS/stored_models/'

In [ ]:
save_json( model, model_storage + "example_keras.json" )

In [ ]:
model = load_model(  model_storage + "example_keras.json" )

In [ ]:
model.save_weights(model_storage + "example_keras.h5")

In [ ]:
model.load_weights(model_storage + "example_keras.h5")

## Implement your own/ Custumize metrics  